In [1]:
# Import findspark and initialize. 
import findspark
findspark.init()

In [2]:
pip install findspark

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install pyspark

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install install-jdk

Note: you may need to restart the kernel to use updated packages.


In [5]:
# Import packages
from pyspark.sql import SparkSession
from pyspark.sql.functions import avg
from pyspark.sql import Row
from pyspark.sql.types import StructType,StructField,StringType, DateType,IntegerType
import time
import jdk

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

24/02/20 12:43:13 WARN Utils: Your hostname, Mrityunjays-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.4.30 instead (on interface en0)
24/02/20 12:43:13 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/02/20 12:43:14 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [6]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"


In [7]:
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=",", header=True)


24/02/20 12:43:24 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [8]:
# 2. Create a temporary view of the DataFrame.
df.createOrReplaceTempView('home_sales')
df.show()

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [9]:
# 3. What is the average price for a four bedroom house sold per year, rounded to two decimal places?
avg = """
SELECT
  YEAR(date),
  ROUND(AVG(price), 2) AS price
FROM home_sales
WHERE bedrooms = 4
GROUP BY YEAR(date)
ORDER BY YEAR(date)
"""
spark.sql(avg).show(4)

+----------+---------+
|year(date)|    price|
+----------+---------+
|      2019| 300263.7|
|      2020|298353.78|
|      2021|301819.44|
|      2022|296363.88|
+----------+---------+



In [10]:
# 4. What is the average price of a home for each year the home was built,
# that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?
bedroom_avg = """SELECT
                ROUND(AVG(price), 2) AS price,
                YEAR(date_built)
                FROM home_sales
                WHERE bedrooms = 3 and
                bathrooms = 3
                GROUP BY YEAR(date_built)

    """
spark.sql(bedroom_avg).show()


+---------+----------------+
|    price|year(date_built)|
+---------+----------------+
| 288770.3|            2015|
|295962.27|            2013|
|290852.27|            2014|
|293683.19|            2012|
|290555.07|            2016|
|292859.62|            2010|
|291117.47|            2011|
|292676.79|            2017|
+---------+----------------+



In [11]:
# 5. What is the average price of a home for each year the home was built,
# that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet, rounded to two decimal places?
spark.sql(""" 
 SELECT
 ROUND(AVG(price), 2) AS price,
 YEAR(date_built) AS year
 FROM home_sales
 WHERE bedrooms = 3 and
 bathrooms = 3 and 
 floors = 2 and
 sqft_lot >= 2000
 GROUP BY YEAR(date_built)
                       
""").show()

+---------+----+
|    price|year|
+---------+----+
|291788.36|2015|
|295142.13|2013|
|294195.13|2014|
|295858.68|2012|
| 287812.6|2016|
|280447.23|2010|
|281413.45|2011|
|282026.59|2017|
+---------+----+



In [13]:
# 6. What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating. 
# Although this is a small dataset, determine the run time for this query.

start_time = time.time()

spark.sql("""select  ROUND(AVG(price), 2) AS price,
          view
          FROM home_sales
          GROUP BY view
          having ROUND(AVG(price), 2) >=  350000
          Order by view desc """).show()

print("--- %s seconds ---" % (time.time() - start_time))

+----------+----+
|     price|view|
+----------+----+
|1061201.42|  99|
|1053739.33|  98|
|1129040.15|  97|
|1017815.92|  96|
| 1054325.6|  95|
| 1033536.2|  94|
|1026006.06|  93|
| 970402.55|  92|
|1137372.73|  91|
|1062654.16|  90|
|1107839.15|  89|
|1031719.35|  88|
| 1072285.2|  87|
|1070444.25|  86|
|1056336.74|  85|
|1117233.13|  84|
|1033965.93|  83|
| 1063498.0|  82|
|1053472.79|  81|
| 991767.38|  80|
+----------+----+
only showing top 20 rows

--- 0.2534778118133545 seconds ---


In [16]:
# 7. Cache the the temporary table home_sales.
spark.sql("cache table home_sales")

DataFrame[]

In [17]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [18]:
# 9. Using the cached data, run the last query above, that calculates 
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000. 
# Determine the runtime and compare it to the uncached runtime.

start_time = time.time()

spark.sql("""select  ROUND(AVG(price), 2) AS price,
          view
          FROM home_sales
          GROUP BY view
          having ROUND(AVG(price), 2) >=  350000
          Order by view desc """).show()



print("--- %s seconds ---" % (time.time() - start_time))


+----------+----+
|     price|view|
+----------+----+
|1061201.42|  99|
|1053739.33|  98|
|1129040.15|  97|
|1017815.92|  96|
| 1054325.6|  95|
| 1033536.2|  94|
|1026006.06|  93|
| 970402.55|  92|
|1137372.73|  91|
|1062654.16|  90|
|1107839.15|  89|
|1031719.35|  88|
| 1072285.2|  87|
|1070444.25|  86|
|1056336.74|  85|
|1117233.13|  84|
|1033965.93|  83|
| 1063498.0|  82|
|1053472.79|  81|
| 991767.38|  80|
+----------+----+
only showing top 20 rows

--- 0.13668394088745117 seconds ---


In [19]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data 
df.write.partitionBy('date_built').parquet('p_home_sales',mode='overwrite')


In [22]:
# 11. Read the formatted parquet data.
p_df=spark.read.parquet('p_home_sales')

In [30]:
# 12. Create a temporary table for the parquet data.
p_df.createOrReplaceTempView('p_home_sales')

In [34]:
# 13. Using the parquet DataFrame, run the last query above, that calculates 
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000. 
# Determine the runtime and compare it to the cached runtime.

start_time = time.time()

spark.sql("""select  ROUND(AVG(price), 2) AS price,
          view
          FROM p_home_sales
          GROUP BY view
          having ROUND(AVG(price), 2) >=  350000
          Order by view desc """).show()



print("--- %s seconds ---" % (time.time() - start_time))

+----------+----+
|     price|view|
+----------+----+
|1061201.42|  99|
|1053739.33|  98|
|1129040.15|  97|
|1017815.92|  96|
| 1054325.6|  95|
| 1033536.2|  94|
|1026006.06|  93|
| 970402.55|  92|
|1137372.73|  91|
|1062654.16|  90|
|1107839.15|  89|
|1031719.35|  88|
| 1072285.2|  87|
|1070444.25|  86|
|1056336.74|  85|
|1117233.13|  84|
|1033965.93|  83|
| 1063498.0|  82|
|1053472.79|  81|
| 991767.38|  80|
+----------+----+
only showing top 20 rows

--- 0.27284908294677734 seconds ---


In [37]:
# 14. Uncache the home_sales temporary table.
spark.sql("uncache table p_home_sales")

DataFrame[]

In [38]:
# 15. Check if the home_sales is no longer cached

spark.catalog.isCached("p_home_sales")

False